
# FX eSSVI — Step-by-step data preparation from article quotes

This notebook shows, **from scratch**, how to turn the Medium article's FX quotes (ATM, 25Δ Put/Call, and rd/rf term structures) into the
training points needed for an **eSSVI** calibration: **$(T, k, w)$** where $k=\ln(K/F)$ and $w=\sigma^2 T$.

- Abhyankar, A. (Medium): *FX Volatility Surface Construction using the Vanna–Volga approach with Python*.
  https://abhyankar-ameya.medium.com/fx-volatility-surface-construction-using-the-vanna-volga-approach-with-python-a8b8a6764686

We will:
1. **Load** the article arrays (tenors $T$, vol quotes, domestic/foreign rates, spot).
2. Compute **discount factors** and the **correct forward** $F(T)$.
3. **Reconstruct strikes** from 25Δ deltas using the **premium-included forward-delta** convention.
4. Convert to **log-moneyness** $k$ and **total variance** $w$.
5. Run **sanity checks** (calendar monotonicity, strike ordering) and **save** a CSV for calibration.

## 1) Load data

In [ ]:
import numpy as np
import pandas as pd
from math import exp
from scipy.stats import norm

# Raw arrays copied from the Medium FX Vanna–Volga post
T = np.array([0.0194,0.04166,0.0833,0.1666,0.25,0.3333,0.4166,0.5,0.75,1,1.25,1.5,2,3,4,5], dtype=float)

Vol_25D_PUT  = np.array([0.121,0.1215,0.1105,0.113,0.1224,0.1236,0.125,0.116,0.1175,0.1322,0.136,0.14,0.1411,0.1433,0.1445,0.145], dtype=float)
Vol_ATM      = np.array([0.118,0.1182,0.1015,0.1029,0.115,0.116,0.118,0.105,0.108,0.121,0.124,0.132,0.135,0.1375,0.14,0.141], dtype=float)
Vol_25D_CALL = np.array([0.1205,0.12,0.115,0.109,0.1125,0.121,0.119,0.108,0.116,0.1275,0.131,0.133,0.1388,0.14,0.1405,0.139], dtype=float)

rd_input = np.array([0.005,0.0052,0.0059,0.006,0.0063,0.0069,0.007,0.0072,0.0075,0.0077,0.008,0.0085,0.009,0.00925,0.0095,0.0098], dtype=float)
rf_input = np.array([0.0043,0.004,0.005,0.0055,0.0068,0.0071,0.0066,0.0078,0.0085,0.0083,0.0088,0.0079,0.0082,0.0087,0.0093,0.0095], dtype=float)

S0 = 1.5  # spot quoted in the article

data = pd.DataFrame({
    "T": T,
    "Vol_25P": Vol_25D_PUT,
    "Vol_ATM": Vol_ATM,
    "Vol_25C": Vol_25D_CALL,
    "rd": rd_input,
    "rf": rf_input,
    "S0": S0
})
data.head()


## 2) Discount factors and correct forwards

For FX forwards we use:
- **Discount factors**: $F(T)=S_0 \dfrac{P_d(0,T)}{P_f(0,T)}$ with $P_{\{d,f\}}(0,T)=e^{-r_{\{d,f\}}T}$.

In [ ]:
# Discount factors
Pd = np.exp(-data["rd"].values * data["T"].values)
Pf = np.exp(-data["rf"].values * data["T"].values)

# Forwards via discount factors
F_df    = S0 * (Pd / Pf)

# Assemble a small table
forwards = pd.DataFrame({
    "T": data["T"],
    "Pd": Pd,
    "Pf": Pf,
    "F_via_df": F_df,
})
forwards.head(10)


## 3) Reconstruct strikes from 25Δ quotes (premium-included forward delta)

Premium-included (PI) forward delta scales by $e^{r_f T}$:
$$\Delta_C^{F,PI}=e^{r_f T}N(d_1),\quad \Delta_P^{F,PI}=e^{r_f T}(N(d_1)-1),
\quad d_1=\frac{\ln(F/K)+\tfrac12\sigma^2T}{\sigma\sqrt{T}}.$$

Invert to get $K$ from a given $\delta$ and $\sigma$:
$$K_C=F\exp\!\big(-\sigma\sqrt{T}N^{-1}(\delta e^{r_f T})+\tfrac12\sigma^2T\big),\quad
K_P=F\exp\!\big(-\sigma\sqrt{T}N^{-1}(1-\delta e^{r_f T})+\tfrac12\sigma^2T\big).$$

ATM (delta-neutral straddle) uses
$$K_{\text{ATM}}=F\,\exp\!\big(\tfrac12\sigma_{\text{ATM}}^2T\big).$$


In [ ]:
from scipy.stats import norm
import numpy as np
import pandas as pd

# Choose the forward (they match): use F_df
F = F_df
T_arr = data["T"].values
rf = data["rf"].values

sigma_ATM = data["Vol_ATM"].values
sigma_25P = data["Vol_25P"].values
sigma_25C = data["Vol_25C"].values

delta = 0.25
scale = np.exp(rf * T_arr)  # e^{r_f T}

def K_call_from_delta(F, T, sigma, delta, scale):
    z = norm.ppf(delta * scale)
    return F * np.exp(-sigma*np.sqrt(T)*z + 0.5*(sigma**2)*T)

def K_put_from_delta(F, T, sigma, delta, scale):
    z = norm.ppf(1.0 - delta * scale)
    return F * np.exp(-sigma*np.sqrt(T)*z + 0.5*(sigma**2)*T)

#K_ATM = F * np.exp(0.5 * (sigma_ATM**2) * T_arr)    # ATM-DN (Delta-neutral) convention
K_ATM = F # ATMF convention (at-the-money-forward)
K_25C = K_call_from_delta(F, T_arr, sigma_25C, delta, scale)
K_25P = K_put_from_delta(F, T_arr, sigma_25P, delta, scale)

pd.DataFrame({
    "T": T_arr, "F": F, "K_25P": K_25P, "K_ATM": K_ATM, "K_25C": K_25C
}).head(10)


## 4) Check for calendar monotonicity of ATM total variance
We verify that the ATM total variance $\theta(T) = \sigma_{\text{ATM}}^2 T$ is **non-decreasing** with maturity.

In [ ]:
# Calendar monotonicity of ATM total variance
theta = (data["Vol_ATM"]**2) * data["T"]
theta_diff = np.diff(theta)
monotone_flags = theta_diff >= -1e-8  # element-wise check
print("theta (ATM total variance):\n", theta.round(6).to_list())
print("\ndiff(theta):\n", theta_diff.round(6).tolist())
print("\nIs non-decreasing step-by-step?\n", monotone_flags.tolist())
print("\nOverall monotone:", np.all(monotone_flags))